This notebook demonstrates how to use our deep learning models to predict recurrence in cancer patients using time-series data. Due to the sensitivity of medical data, we are unable to release the real dataset. Instead, we demonstrate the use of our models on synthetic time-series dataset

In [65]:
# Import required packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, Input, Flatten, Concatenate, RepeatVector, TimeDistributed, Conv1D, Layer, LayerNormalization, GlobalAveragePooling1D, LSTM
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K 
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Import dataset.  
Our method for generating synthetic data is described in https://github.com/phu5ion/colorectal-prognostication/tree/master/notebooks/create_simul_data.ipynb. However any method can be used as long as synthetic data is generated.

In [18]:
ts = pd.read_csv("synthetic_ts.csv")
ts.head()

,timestamps,cea,patient_id
0,2020-08-01,0.359474,1
1,2020-09-01,0.485895,1
2,2020-10-01,0.871115,1
3,2020-10-31,0.871115,1
4,2020-12-01,1.663848,1


In [29]:
# Normalise CEA values
cea = ts['cea'].to_numpy().reshape(-1, 1)
s = MinMaxScaler()
out = s.fit_transform(cea)
ts['normed'] = out
ts.head()

,timestamps,cea,patient_id,normed
0,2020-08-01,0.359474,1,0.034344
1,2020-09-01,0.485895,1,0.047310
2,2020-10-01,0.871115,1,0.086819
3,2020-10-31,0.871115,1,0.086819
4,2020-12-01,1.663848,1,0.168124


In [42]:
# Next reshape into list of normalised CEA
out = []
patrow = []
patid = 1
for row in ts.iterrows():
    pat = row[1]['patient_id']
    cea = row[1]['normed']
    if pat != patid:
        out.append(patrow)
        patrow = []
        patid = pat
    else:
        patrow.append(cea)
out.append(patrow) # Append last row
print(len(out)) # 900 patients
    

900


In [84]:
# Then we pad the dataset
dataset = tf.keras.preprocessing.sequence.pad_sequences(out, maxlen=None, dtype='float64', padding='pre', truncating='pre')
dataset = np.expand_dims(dataset, axis=-1) # Because we only have 1 feature, we add an extra dimension to fit what model expects
print(dataset.shape)
ts_length = dataset.shape[1]

(900, 32, 1)


In [85]:
# Get labels from the tabular data
tabular = pd.read_csv("synthetic_structured.csv")
labels = tabular['relapse'].to_numpy()
labels = np.where(labels=='yes', 1, 0)

In [86]:
# Do train-val-test split
xtrain, xtest, ytrain, ytest = train_test_split(dataset, labels, test_size=0.2, random_state=100)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size=0.2, random_state=100)
xtrain.shape, xval.shape, xtest.shape

((576, 32, 1), (144, 32, 1), (180, 32, 1))

# Import models

In [55]:
# %load models/htmv.py
# Code for positional encodings
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

class MultiHeadSelfAttention(Layer):
    def __init__(self, embed_dim, num_heads=8, future_mask=None, use_cnn=True):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads # query, value and key vector dim will be embedding dim // num_heads
         
        if use_cnn:
            self.query_dense = Conv1D(embed_dim, kernel_size=3, padding="causal")
            self.key_dense = Conv1D(embed_dim, kernel_size=3, padding="causal")
            self.value_dense = Conv1D(embed_dim, kernel_size=3, padding="causal")
        else:
            self.query_dense = Dense(embed_dim)
            self.key_dense = Dense(embed_dim)
            self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)
        self.future_mask = future_mask
    def attention(self, query, key, value, mask=None):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        if mask is not None:
            scaled_score += (mask * -1e9)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3]) # transposes along dimension 0 (the batch)

    def call(self, inputs): # Query, key and value are the input tensors to attention model
        # x.shape = [batch_size, seq_len, embedding_dim]
        query_input, key_input, value_input = inputs
        batch_size = tf.shape(query_input)[0]
        # Initialise query, key and value vectors
        query = self.query_dense(query_input)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(key_input)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(value_input)  # (batch_size, seq_len, embed_dim)
        # Separate out an extra dim by num_heads. So calculation is done separately for each head. 
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        # Perform the attention calculation
        attention, weights = self.attention(query, key, value, self.future_mask)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim) # Do the transpose to make it easier to combine the last two dimensions later on 
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim) # embed_dim = projection_dim * num_heads
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim) # do a linear projection
        return output
def transformer(d_model, ts_length, stack=2, h=8, dropout=0.1, local_att_size=10, mask='local', use_cnn=True):
    # stack: num encoder and decoder stacks, each
    # h: num heads. h>1 for multi-headed attention
    kernel_init='he_uniform'
    
    ts_input = Input(shape=(ts_length, 1))
    # Positional encodings
    pos_encodings = positional_encoding(ts_length, d_model)
    # Embedding
    embedding = Dense(d_model, activation='linear', name='original_encodings')(ts_input + pos_encodings) # Linear encoding # Dense layer with 1dim is actually the same as an Embedding layer
    # Create look_ahead_mask for causal attention. This works on the global sequence
    seq_len = ts_length
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0) # Set lower triangular part to 1, upper to 0
    # We also create a local mask that also implements causal attention so that it doesn't need to look so far behind
    local_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), local_att_size, 0)
    # We train this as a stack of attentional layers with causal masking
    x = embedding
    
    # Which mask to use?
    if mask=='local':
        mask = local_mask
    else:
        mask = look_ahead_mask
    # Encoder
    for i in range(stack):
        # This is 1 transformer block
        # Self attention
        residual = x
        x = MultiHeadSelfAttention(embed_dim=d_model, num_heads=h, future_mask=None, use_cnn=use_cnn)([x,x,x]) # Passing in list of query, key and value inputs for self-attention
        x = Dropout(dropout)(x)
        x = LayerNormalization(epsilon=1e-6)(x+residual)
        # Feed-forward
        residual = x
        x = Sequential([Dense(d_model, activation="relu", kernel_initializer=kernel_init), Dense(d_model),])(x) # feed-forward layer with relu in between
        x = Dropout(dropout)(x)
        x = LayerNormalization(epsilon=1e-6)(x+residual)
    enc_output = x # We save this in our memory
    # Decoder
    for i in range(stack):
        # This is 1 transformer block
        # Self attention
        residual = x
        x = MultiHeadSelfAttention(embed_dim=d_model, num_heads=h, future_mask=mask, use_cnn=use_cnn)([x,x,x]) 
        x = Dropout(dropout)(x)
        x = LayerNormalization(epsilon=1e-6)(x+residual)
        # Encoder-decoder attention
        residual = x
        x = MultiHeadSelfAttention(embed_dim=d_model, num_heads=h, use_cnn=use_cnn)([x,enc_output,enc_output]) # key and value are our encoder output/memory
        x = Dropout(dropout)(x)
        x = LayerNormalization(epsilon=1e-6)(x+residual)
        # Feed-forward
        residual = x
        x = Sequential([Dense(d_model, activation="relu", kernel_initializer=kernel_init), Dense(d_model),])(x) # feed-forward layer with relu in between
        x = Dropout(dropout)(x)
        x = LayerNormalization(epsilon=1e-6)(x+residual)
        
    # Apply dense layer
    x = GlobalAveragePooling1D()(x)
    x = Dense(20, activation='relu', kernel_initializer=kernel_init)(x)
    outputs = Dense(1, activation="sigmoid", )(x)

    model = Model(inputs=ts_input, outputs=outputs)
    return model


In [108]:
# %load models/dl_models.py
# LSTM
def create_masking_model(ts_length, bidir=1):
    ts_input = Input((ts_length, 1))
    kernel_regularizer=L1L2(l1=0.01, l2=0.01)
    # LSTM-input
    mask = tf.keras.layers.Masking(mask_value=0.0)(ts_input)
    if bidir:
        lstm_layer=Bidirectional(LSTM(8, return_sequences=True, dropout = 0.2, recurrent_dropout=0.2, activation='tanh', kernel_regularizer=kernel_regularizer, kernel_initializer="glorot_uniform"))(mask)
    else:
        lstm_layer=LSTM(16, return_sequences=True, dropout = 0.2, recurrent_dropout=0.2, activation='tanh', kernel_regularizer=kernel_regularizer, kernel_initializer="glorot_uniform")(mask)
    lstm_layer=LSTM(8, dropout = 0.2, activation='tanh', recurrent_dropout=0.2, kernel_regularizer=kernel_regularizer, kernel_initializer="glorot_uniform")(lstm_layer)
    output=Dense(1, activation='sigmoid')(lstm_layer)
    # define a model with a list of two inputs
    model = Model(inputs=ts_input, outputs=output)
    return model

# Temporal Convolutional Network (TCN aka Wavenet)
def create_tcn(num_channels, ts_length, kernel_size=2, strides=1, dropout=0.1):
    # Initialise required stuff
    ts_input = Input(shape=(ts_length, 1))
    kernel_regularizer=L1L2(l1=0.00, l2=0.00)
    kernel_initializer="he_uniform" # Instead of glorot_uniform as he_uniform seems to be theoretically better for relu and relu-like activations
    lnorm=LayerNormalization() # LayerNorm and BatchNorm both doesn't work...?
    # Depending on the number of levels, we increase dilation rate.
    # Number of levels should be self-calculated...
    num_levels = len(num_channels) # It should look like a list of length levels, how many filters each level
    inputs = ts_input
    for i in range(num_levels):
        dilation_size = 2 ** i
        out_channels = num_channels[i]
        
        # This is 1 block
        cnn1 = Conv1D(filters=out_channels, kernel_size=kernel_size, strides=strides, padding='causal', data_format='channels_last', activation='relu', dilation_rate=dilation_size, kernel_initializer=kernel_initializer)(inputs)
        #norm1 = lnorm(cnn1)
        dropout1 = Dropout(dropout, noise_shape=[tf.constant(1), tf.constant(1), tf.constant(out_channels)])(cnn1) # Noise_shape is to apply uniform dropout to all timesteps
        cnn2 = Conv1D(filters=out_channels, kernel_size=kernel_size, strides=strides, padding='causal', data_format='channels_last', activation='relu', dilation_rate=dilation_size, kernel_initializer=kernel_initializer)(dropout1)
        #norm2 = lnorm(cnn2)
        dropout2 = Dropout(dropout, noise_shape=[tf.constant(1), tf.constant(1), tf.constant(out_channels)])(cnn2) # Noise_shape is to apply uniform dropout to all timesteps
        out = relu(dropout2 + inputs) # Skip connections
        inputs = out
            
    out = out[:, -1, :]
    output = Dense(1, activation='sigmoid')(out)
    
    # define a model with a list of two inputs
    model = Model(inputs=ts_input, outputs=output)
    return model


In [73]:
# Define some metrics
auc = tf.keras.metrics.AUC()
prec = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
trueneg = tf.keras.metrics.TrueNegatives()
# Implement my own Balanced Acc calculation
def balanced_acc(y_true, y_pred):
    def recall(y_true, y_pred):
        # recall of class 1

        #do not use "round" here if you're going to use this as a loss function
        true_positives = K.sum(K.round(y_pred) * y_true)
        possible_positives = K.sum(y_true)
        return true_positives / (possible_positives + K.epsilon())

    return (recall(y_true,y_pred) + recall(1-y_true,1-y_pred))/2.

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

In [110]:
def train_model(modelname, xtrain, xval, ytrain, yval, batch_size, epoches, filename, verbose=1): 
    if modelname == 'transformer':
        model = transformer(d_model=64, ts_length=ts_length, stack=1, dropout=0.1, h=8, local_att_size=10)
    elif modelname == 'lstm':
        model = create_masking_model(ts_length=ts_length, bidir=1)
    elif modelname == 'tcn':
        layers=6
        model = create_tcn([60]*layers, ts_length=ts_length, kernel_size=2, dropout=0.1) # 60 hidden nodes, 6 levels
    opt = Adam(clipvalue=5, lr=0.0001) 
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[balanced_acc, recall, prec, specificity])
    history = model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epoches, verbose=verbose, validation_data=[xval, yval])
    # Save model
    model.save_weights(filename+"/"+filename+".ckpt")

In [114]:
train_model('tcn', xtrain, xval, ytrain, yval, batch_size=32, epoches=100, filename='test-tcn', verbose=1)

Train on 576 samples, validate on 144 samples
Epoch 1/100
576/576 [==============================] - 3s 6ms/sample - loss: 1.6952 - balanced_acc: 0.5077 - recall: 0.9315 - precision: 0.2731 - specificity: 0.1384 - val_loss: 0.5782 - val_balanced_acc: 0.5000 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_specificity: 1.0000
Epoch 2/100
576/576 [==============================] - 0s 801us/sample - loss: 0.6590 - balanced_acc: 0.5102 - recall: 0.1233 - precision: 0.3051 - specificity: 0.9019 - val_loss: 0.6040 - val_balanced_acc: 0.5000 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_specificity: 1.0000
Epoch 3/100
576/576 [==============================] - 0s 779us/sample - loss: 0.6579 - balanced_acc: 0.5092 - recall: 0.1438 - precision: 0.2763 - specificity: 0.8746 - val_loss: 0.6207 - val_balanced_acc: 0.5000 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_specificity: 1.0000
Epoch 4/100
576/576 [==============================] - 0s 783us/sample - loss

In [115]:
def test_model(xtest, ytest, modelname, modeldir):
     # Initialise trained model
    if modelname == 'transformer':
        newmodel = transformer(d_model=64, ts_length=ts_length, stack=1, dropout=0.1, h=8, local_att_size=10)
    elif modelname == 'lstm':
        newmodel = create_masking_model(ts_length=ts_length, bidir=1)
    elif modelname == 'tcn':
        layers=6
        newmodel = create_tcn([60]*layers, ts_length=ts_length, kernel_size=2, dropout=0.1) # 60 hidden nodes, 6 levels
    # Restore the weights
    filename = modeldir+'/'+modeldir+'.ckpt'
    newmodel.load_weights(filename).expect_partial()
    
    y_proba = newmodel.predict(xtest).squeeze()
    y_preds = np.round(y_proba)
    acc = metrics.accuracy_score(y_true=ytest, y_pred=y_preds)
    recall = metrics.recall_score(y_true=ytest, y_pred=y_preds)
    precision = metrics.precision_score(y_true=ytest, y_pred=y_preds)
    f1 = metrics.f1_score(y_true=ytest, y_pred=y_preds)
    bal_acc = metrics.balanced_accuracy_score(y_true=ytest, y_pred=y_preds)
    tn, fp, fn, tp = metrics.confusion_matrix(y_true=ytest, y_pred=y_preds).ravel()
    specificity = tn / (tn+fp)
    aucpr = metrics.average_precision_score(y_true=ytest, y_score=y_proba)
    print(f'Acc: {acc}, Recall: {recall}, Precision: {precision}, Specificity: {specificity}') # You can change whichever metrics you want to be printed out here 
test_model(xtest, ytest, 'tcn', 'test-tcn')

Acc: 0.7333333333333333, Recall: 0.022222222222222223, Precision: 0.2, Specificity: 0.9703703703703703
